In [1]:
import pandas as pd
df = pd.read_csv('../csv_data/Radionics/Radionics_v1.csv')

df

,TEXT,X1,Y1,X2,Y2,block_no,line_no,page_no
0,Radionics,109,26,137,34,0,0,1
1,"Ltd,",138,26,150,34,0,0,1
2,Glenview,109,34,136,42,0,1,1
3,Industrial,138,34,164,42,0,1,1
4,"Estate,",166,34,185,42,0,1,1
...,...,...,...,...,...,...,...,...
299,&,328,760,334,768,10,6,1
300,service,336,760,355,768,10,6,1
301,available,357,760,382,768,10,6,1
302,on,384,760,391,768,10,6,1


In [11]:
from pdf2df import convert_to_df
df = convert_to_df('../pdf_issue/radionics_1.pdf')


{'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'SPS APLOAD version 3.2.0', 'producer': 'SPS APLOAD version 3.2.0', 'creationDate': 'D:20220830065528Z', 'modDate': "D:20220906102840+01'00'", 'trapped': '', 'encryption': None}


Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.


In [2]:
def find_text(temp_df, text,case=False):
    location = temp_df[temp_df['TEXT'].str.contains(text,case=case, na=False)].index.tolist()
    return location

def find_next_word(temp_df, text, nearest_text, case=False):
    pos = find_text(temp_df,text,case=case)
    pos = [item +1 for item in pos]
#     print(pos)
    new_pos = list()
    for item in pos:
#         temp = df.loc[]
        temp_value= temp_df.loc[item]['TEXT']
#         print(temp_value.lower())
       
        if temp_value.lower().__contains__(nearest_text.lower()):
            new_pos.append(item-1)
        
    return new_pos

In [3]:
pos = find_next_word(df,'Invoice', 'No.')
invoice_no = df[pos[0]+2:pos[0]+3]['TEXT'].values[0]
invoice_no

'413306129'

In [4]:
pos_date =find_text(df,'date')[0]
date = df[pos_date+3:pos_date+4]['TEXT'].values[0]
date

'29.08.2022'

In [12]:
table_start = find_next_word(df,'qty','sales')
print(table_start)
table_end = find_text(df, 'Standard')
print(table_end)
table_data = df[table_start[0]:table_end[-1]]
table_data.to_csv('../pdf_issue/radionics_1.csv')

[112]
[176]


In [27]:
find_text(df,'EA')

[131, 187, 209, 301, 525]

In [28]:
df[130:140]

,TEXT,X1,Y1,X2,Y2,block_no,line_no,page_no
130,10,304,451,312,462,8,3,1
131,EA,314,451,325,462,8,3,1
132,10,415,451,423,462,8,4,1
133,1.001,461,451,479,462,8,5,1
134,10.01,518,451,536,462,8,6,1
135,23.00%,543,451,567,462,8,7,1
136,HS,111,461,121,472,8,8,1
137,export,123,461,144,472,8,8,1
138,code,146,461,161,472,8,8,1
139,85361050,163,461,195,472,8,8,1


In [14]:
from pprint import pprint
msg = dict()
result = list()
for index, row in table_data.iterrows():
    
    if row['X1'] < 39:
        try:
           
            msg = dict()
            msg['item'] = int(row['TEXT'].strip())
            if msg['item'] :
                msg['part'] = ''
            
        except Exception as e:
            print('its no working')
        
    elif ('part' in msg) and (50 < row['X1'] < 87):
        msg['part']= row['TEXT']
        msg['desc']= ''

        
    elif ('desc' in msg) and (100 < row['X1'] < 250):
        msg['desc'] = msg['desc'] +" "+ row['TEXT']
        msg['qty'] = ''

    elif ('qty' in msg) and (280 < row['X1'] < 310):
        print(row['TEXT'])
        msg['qty'] = row['TEXT']
        msg['unit'] = 0
        print(msg)
        
    elif ('unit' in msg) and (450  < row['X1'] < 479):
        msg['unit'] = row['TEXT']
        msg['total'] = 0
        
    elif ('total' in msg) and (500 < row['X1'] < 530):
        msg['total'] = row['TEXT']
        pprint(msg)
        result.append(msg)
        msg=dict()
        
        

200
{'item': 10, 'part': '534446', 'desc': ' Yel M5 crimp ring terminal,2.5-6.5sq.mm', 'qty': '200', 'unit': 0}
{'desc': ' Yel M5 crimp ring terminal,2.5-6.5sq.mm',
 'item': 10,
 'part': '534446',
 'qty': '200',
 'total': '49.38',
 'unit': '0.247'}
1
{'item': 30, 'part': '8399875', 'desc': ' Hand Held Proximity Switch Tester', 'qty': '1', 'unit': 0}
{'desc': ' Hand Held Proximity Switch Tester',
 'item': 30,
 'part': '8399875',
 'qty': '1',
 'total': '86.76',
 'unit': '86.76'}
